In [1]:
from pprint import pprint
from lxml import html
import requests
from pymongo import MongoClient

### 1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news.
#### Для парсинга использовать xpath. Структура данных должна содержать: 
##### 1) название источника,
##### 2) наименование новости,
##### 3) ссылку на новость,
##### 4) дата публикации
### 2) Записать данные в БД.

### news.mail.ru

In [5]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['news_db'] # Создаем БД
mail = db.mail # Создаем объект, который создаст коллекцию mail в БД news_db

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

def request_to_m_news():
    response = requests.get('https://news.mail.ru/',
                            headers=header)
    dom = html.fromstring(response.text)
    
    # Наименование новости
    
    name = dom.xpath("//div[contains(@class,'daynews__item')]//span[contains(@class,'photo__title')]/text()")
    
    # Ссылка на новость
    
    link = dom.xpath("//a[contains(@class,'photo photo')]/@href")
    
    for i in range(len(name)):
        
        # Источник
        
        header1 = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}
        
        response1 = requests.get('https://news.mail.ru/'+link[i],
                            headers=header1)
        dom1 = html.fromstring(response1.text)
        
        istochnik = dom1.xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()")
        
        # Дата
        
        data = dom1.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
                
        print(i+1, 'Источник:', str(istochnik[0]), 'Наименование новости:', name[i], 'Ссылка на новость:', ('https://news.mail.ru/'+link[i]), 'Дата:', str(data[0]))

        mail.insert_one({"Источник": str(istochnik[0]),
                "Наименование новости": name[i],
                "Ссылка на новость": ('https://news.mail.ru/'+link[i]),
                "Дата": str(data[0])})
        

request_to_m_news()

for i in mail.find():
    pprint(i)

1 Источник: РИА Новости Наименование новости: Путин рассказал о недостатках в Конституции СССР Ссылка на новость: https://news.mail.ru//politics/42452596/ Дата: 2020-07-05T14:23:53+03:00
2 Источник: Комсомольская правда Наименование новости: Россиянам рассказали о синдроме «постковидного лета» Ссылка на новость: https://news.mail.ru//society/42449551/ Дата: 2020-07-05T15:20:12+03:00
3 Источник: Коммерсантъ Наименование новости: Экс-премьер ДНР: республики Донбасса скоро станут частью России Ссылка на новость: https://news.mail.ru//politics/42450563/ Дата: 2020-07-05T13:32:58+03:00
4 Источник: ТАСС Наименование новости: Четыре человека погибли при обрушении перекрытий строящегося ТЦ Ссылка на новость: https://news.mail.ru//incident/42451656/ Дата: 2020-07-05T11:59:20+03:00
5 Источник: Новости Mail.ru Наименование новости: Актер Вилле Хаапасало стал продавать хачапури (фото, видео) Ссылка на новость: https://news.mail.ru//society/42452968/ Дата: 2020-07-05T14:30:52+03:00
{'_id': ObjectId

### lenta.ru

In [6]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['news_db'] # Создаем БД
lenta = db.lenta # Создаем объект, который создаст коллекцию lenta в БД news_db

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

def request_to_lenta():
    response = requests.get('https://lenta.ru/',
                            headers=header)
    dom = html.fromstring(response.text)
    
    # Наименование новости
    
    name1 = dom.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[contains(@class,'item')]//a[contains(@href,'news')]/text()")
    #pprint(name)
    # Ссылка на новость
    
    link1 = dom.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[contains(@class,'item')]//a[contains(@href,'news')]/@href")
    #pprint(link)
    
    # Дата
        
    data1 = dom.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[contains(@class,'item')]//a[contains(@href,'news')]/time[@class='g-time']/@datetime")
    #pprint(data1)
        
    for i in range(len(name1)):
                                 
        print(i+1, 'Источник: lenta.ru', 'Наименование новости:', name1[i], 'Ссылка на новость:', ('https://lenta.ru/'+link1[i+1]), 'Дата:', data1[i])
    
        
        lenta.insert_one({"Источник": 'lenta.ru',
                "Наименование новости": name1[i],
                "Ссылка на новость": ('https://lenta.ru/'+link1[i+1]),
                "Дата": data1[i]})
        
        
request_to_lenta()

for i in lenta.find():
    pprint(i)

1 Источник: lenta.ru Наименование новости: Путин рассказал о «мине замедленного действия» в советской Конституции Ссылка на новость: https://lenta.ru//news/2020/07/05/mina/ Дата:  13:27,  5 июля 2020
2 Источник: lenta.ru Наименование новости: Гонщик «Формулы-1» отказался встать на колено в знак борьбы с расизмом Ссылка на новость: https://lenta.ru//news/2020/07/05/max/ Дата:  15:50,  5 июля 2020
3 Источник: lenta.ru Наименование новости: В Госдуме отреагировали на планы Канье Уэста стать президентом США Ссылка на новость: https://lenta.ru//news/2020/07/05/kanyetrump/ Дата:  15:34,  5 июля 2020
4 Источник: lenta.ru Наименование новости: В российском регионе захотели изменить свою конституцию Ссылка на новость: https://lenta.ru//news/2020/07/05/newconst/ Дата:  15:29,  5 июля 2020
5 Источник: lenta.ru Наименование новости: Москвичкам-скорпионам отказали в услугах косметологии из-за «Меркурия в Раке»  Ссылка на новость: https://lenta.ru/https://moslenta.ru/news/moskvichkam-skorpionam-otka

### yandex.news

In [8]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['news_db'] # Создаем БД
yandex = db.yandex # Создаем объект, который создаст коллекцию yandex в БД news_db


header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

def request_to_yandex():
    response = requests.get('https://yandex.ru/',
                            headers=header)
    dom = html.fromstring(response.text)
    
    # Наименование новости
    
    name2 = dom.xpath("//ol/li[contains(@class,'list__item  list__item_icon')]/a[contains(@class,'home-link list__item-content list__item-content_with-icon home-link_black_yes')]/@aria-label")
    #pprint(name2)
    # Ссылка на новость
    
    link2 = dom.xpath("//ol/li[contains(@class,'list__item  list__item_icon')]/a[contains(@class,'home-link list__item-content list__item-content_with-icon home-link_black_yes')]/@href")
    #pprint(link2)
    
    # Источник
    
    istochnik2 = dom.xpath("//ol/li[contains(@class,'list__item  list__item_icon')]/a[contains(@class,'home-link list__item-content list__item-content_with-icon home-link_black_yes')]//object[@class='news__agency-icon-image']/@title")
    
    
    
    for i in range(len(name2)):
        
        # Дата
        data = dom.xpath("//span[contains(@class,'datetime__day')]/text()") # число
        data1 = dom.xpath("//span[contains(@class,'datetime__month')]/text()") # месяц
        data2 = dom.xpath("//span[contains(@class,'datetime__wday')]/text()")  # день недели
        #data = dom.xpath("//span[contains(@class,'datetime datetime_short')]/span[contains(@class,'datetime__date')]/span[contains(@class,'datetime')]/text()")
        f = str(data[0]+data1[0]+data2[0])
        #print(f)
                
        print(i+1, 'Источник:', istochnik2[i], 'Наименование новости:', name2[i], 'Ссылка на новость:', str(link2[i]), 'Дата:', f)
   
        yandex.insert_one({"Источник": istochnik2[i],
                "Наименование новости": name2[i],
                "Ссылка на новость": str(link2[i]),
                "Дата": f})

request_to_yandex()
                           
for i in yandex.find():
    pprint(i)

1 Источник: BFM.ru Наименование новости: Путин оценил принятые поправки к Конституции России Ссылка на новость: https://yandex.ru/news/story/Putin_ocenil_prinyatye_popravki_k_Konstitucii_Rossii--865504c75a5b5eec13dc39d8fb1b07ce?from=main_portal&lang=ru&lr=213&mlid=1593954565.glob_225.865504c7&msid=1593955101.89864.82734.536302&persistent_id=104735425&stid=XEYR5UPE7O0PWHYT5NH4&t=1593954565&utm_medium=topnews_news&utm_source=morda_desktop&wan=1 Дата: 5 июля, воскресенье 
2 Источник: Газета.Ru Наименование новости: Гонщик «Ред Булла» Ферстаппен объявил об отказе вставать на колено Ссылка на новость: https://yandex.ru/sport/story/Gonshhik_Red_Bulla_Ferstappen_obyavil_ob_otkaze_vstavat_na_koleno--51121042e4a98a6aa283b5d8e9b0f896?from=main_portal&lang=ru&lr=213&mlid=1593954565.glob_225.51121042&msid=1593955101.89864.82734.536302&persistent_id=104679838&stid=M_0bab58GonMyvEaoZpj&t=1593954565&utm_medium=topnews_news&utm_source=morda_desktop&wan=1 Дата: 5 июля, воскресенье 
3 Источник: Известия